In [ ]:
import os

# os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np
import numpy.ma as ma
import os
import time

import pandas as pd
# from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn import mixture, metrics

# from shapely.geometry import Polygon


from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

from skimage.measure import approximate_polygon

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob

# from radio_beam.commonbeam import getMinVolEllipse
#
# from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
# from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset
#
# import plotly.graph_objects as go



from tqdm.notebook import tqdm
import json

import cv2

from collections import Counter

from numba import jit

In [ ]:
{'path': paths[i],
 'name': name,
 'type': types_dict[name],
 'legend': [{'a_beams': legend1, 'b_beams': legend2}],
 'density_curve_scatter': [
     {'a_beams': (x1.flatten(), y1.flatten()),
      'b_beams': (x2.flatten(), y2.flatten()),
      'angles': (x_angles.flatten(), y_angles.flatten())
      }
 ],
 'linear_approx_plot': [{'a_beams': (x_pred1.flatten(), y_pred1.flatten()),
                         'b_beams': (x_pred2.flatten(), y_pred2.flatten())}],
 'linear_approx_data': [{'a_beams': {'k': k1, 'b': b1, 'angle': angle1, 'score': score1},
                         'b_beams': {'k': k2, 'b': b2, 'angle': angle2, 'score': score2}}],
 'beams_length_series': [{'a_beams': all_a_beams, 'b_beams': all_b_beams}],
 'angles_series': all_angles,
 'contours_series': all_contours,
 'pixel2meter': pixel,
 }

In [ ]:
step = 2

# path=f'left_(1500,1500,1)_step_{step}_beams.json'
path = f'bc_left_(1536,1536,1)_preprocess_step_{step}_beams.json'

data = open(path, encoding='utf-8')
data = json.load(data)

In [ ]:
def lin_regr_approx(x, y):
    """
    :param x: list (n,1)
    :param y: list (n,1)
    :return: (x_pred, y_pred), k, b, angle, score
    """
    #
    # аппроксимация распределения линейной функцией и создание графика по параметрам распределения
    #

    x_pred = np.linspace(x.min(axis=0), x.max(axis=0), 50)

    reg = LinearRegression().fit(x, y)
    y_pred = reg.predict(x_pred)

    k = reg.coef_[0][0]
    b = reg.predict([[0]])[0][0]

    angle = np.rad2deg(np.arctan(k))
    score = reg.score(x, y)

    return (x_pred, y_pred), k, b, angle, score

In [ ]:
def stats_preprocess(array, step):
    """
    :param array: list, ndarray (n,)
    :param step: int
    :return: array_copy, array_copy_set, dens_curve
    """
    #
    # приведение углов к кратости, например 0,step,2*step и тд
    # работает только для целых чисел!!!
    #
    new_array = []
    values_dict = {}
    if step != 0:
        for i, a in enumerate(array):
            array_element = array[i]
            new_array_element = array[i]
            val = array_element % step
            if val != 0:
                if val < step / 2:
                    new_array_element = array_element - val
                else:
                    new_array_element = array_element + step - val

            new_array_element=np.round(new_array_element)

            new_array.append(new_array_element)

            if values_dict.get(new_array_element) != None:
                values_dict[new_array_element].append(array_element)
            else:
                values_dict[new_array_element] = []
                values_dict[new_array_element].append(array_element)

        keys = list(values_dict.keys())
        keys.sort()
        sorted_dict = {i: values_dict[i] for i in keys}

        new_array=np.round(new_array)

        cnt = Counter(new_array)
        counts = np.array(list(cnt.items()))
        counts = counts[counts[:, 0].argsort()]

        # polygon_areas, areas_set, areas_dens_curve, areas for each discrete area value
        return np.array(new_array), counts[:, 0], counts[:, 1], sorted_dict
    else:
        print('step is 0, stats preprocess error')

In [ ]:
@jit(nopython=True)
def contourArea(cnt):
    print(cnt)
    area=0
    for i, c in enumerate(cnt[:-1]):
        x,y=c
        x_next,y_next=cnt[i+1]
        area+=x*y_next-y*x_next

    area*=0.5

    return abs(area)

In [ ]:
def polygons_area_all(data, saved_image_name, step, N, M, indices=None, save=False, start=1, end=None, pixel=50 / 1000,
                      font_size=20, s=60, log_min_val=-8, min_area_num=10):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #

    pixel = pow(pixel, 2)
    alloys_indices = range(data['data'].__len__())

    if indices is not None:
        alloys_indices = indices

    fig = plt.figure(figsize=(N, M), facecolor='white')
    fig.patch.set_facecolor('white')
    ax = plt.axes()
    ax.set_facecolor("white")

    plt.rcParams.update({'font.size': font_size})

    markers = ['v', 's', 'D', 'd', 'p', '*', 'P']
    colors = ['orange', 'red', 'blue', 'green', 'indigo']

    legend_a = []

    for k, i in tqdm(enumerate(alloys_indices)):
        cnts = data['data'][i]['contours_series']
        polygon_areas = np.array([contourArea(np.array(cnt)) for cnt in cnts])

        # print('middle')
        images_num = data['data'][i]['linear_approx_plot'][0]['a_beams'][0].__len__()

        co_fraction = np.sum(polygon_areas) / (pow(1535, 2) * images_num)

        polygon_areas, areas_set, areas_dens_curve, areas_dict = stats_preprocess(polygon_areas * pixel, step)

        bins_std={}
        for key in list(areas_dict.keys()):
            areas=np.array(areas_dict[key])

            mean=np.mean(areas)
            std=np.sqrt(np.mean(abs(areas-mean)**2))

            # std=np.std(areas)
            bins_std[key]=std

        yerr=list(bins_std.values())

        areas_dens_curve /= np.sum(areas_dens_curve)
        areas_dens_curve = np.log(areas_dens_curve)

        # убираем те точки и площади, вероятность которых меньше log_min_val
        areas_dens_curve = [h for h in areas_dens_curve if h > log_min_val]
        areas_set = areas_set[:len(areas_dens_curve)]
        yerr=yerr[:len(areas_dens_curve)]
        polygon_areas = [area for area in polygon_areas if area < areas_set[-1]]

        # убираем точки и площади, которых меньше
        areas_dict={key:areas_dict[key] for key in list(areas_dict.keys()) if areas_dict[key].__len__()>min_area_num}
        yerr=yerr[:len(areas_dict)]
        areas_dens_curve = areas_dens_curve[:len(areas_dict)]
        areas_set = areas_set[:len(areas_dict)]

        polygon_areas = [area for area in polygon_areas if area < areas_set[-1]]

        polygon_mean = np.mean(polygon_areas)
        polygon_std = np.std(polygon_areas)

        plt.scatter(areas_set, areas_dens_curve, marker=markers[i], color=colors[i], s=s)

        areas_dens_curve = np.array(areas_dens_curve)
        areas_set = np.array(areas_set)

        (x_pred, y_pred), k, b, angle, score = lin_regr_approx(areas_set[2:35].reshape(-1, 1),
                                                               areas_dens_curve[2:35].reshape(-1, 1))

        plt.plot(x_pred, y_pred, color=colors[i])
        plt.errorbar(areas_set, areas_dens_curve, yerr=yerr,linestyle="None",color=colors[i] )

        legend_a.append(data['data'][i]['name'] + ' ' + data['data'][i][
            'type'] + f'\nсредняя площадь={round(polygon_mean, 2)} $мкм^2$\nstd={np.round(polygon_std, 2)}\nдоля кобальта ={np.round(co_fraction, 2)}'
                      f'\nk= {round(k, 2)}\nscore= {round(score, 2)}\nшаг площади={step}')
        legend_a.append('-------')

    plt.legend(legend_a,fontsize=font_size)
    plt.ylabel('p ( x )')

    plt.xlabel('Площадь регионов кобальта, $мкм^2$')
    # ax2.legend(legend_a, loc=1)

    # plt.xticks(np.arange(0,51,5))
    # ax2.set_xticks(np.arange(0,12,3))
    # plt.xlim(0,50)
    # plt.ylim(0,0.15)
    # plt.ylim(-8,0)
    plt.grid(True)

    if save:
        plt.savefig(f'polygons_area_{saved_image_name}_step_{step}.png', bbox_inches='tight')

    plt.show()

    return bins_std,areas_dict, yerr


In [ ]:
%%time
N = 25
M = 14

save = False
# save=True
step = 1

saved_image_name = f'original'

bins_std,areas_dict,yerr=polygons_area_all(data, saved_image_name, step, N, M, indices=[3,0,2], save=save, start=0, log_min_val=-10,min_area_num=10)
# polygons_area_all(data,saved_image_name, step, N, M, indices=[3,0,2],save=save,end=None)
# polygons_area_all(path,saved_image_name, step, N, M, indices=None,save=save)


In [ ]:
%%time
N = 25
M = 14

save = False
# save=True
step = 1

saved_image_name = f'original'

bins_std,areas_dict,yerr=polygons_area_all(data, saved_image_name, step, N, M, indices=[3,0,2], save=save, start=0, log_min_val=-10,min_area_num=10)
# polygons_area_all(data,saved_image_name, step, N, M, indices=[3,0,2],save=save,end=None)
# polygons_area_all(path,saved_image_name, step, N, M, indices=None,save=save)


In [ ]:
yerr

## Эффективный радиус

In [ ]:
def polygons_radius_all(data, saved_image_name, step, N, M, indices=None, save=False, start=1, end=None,
                        pixel=50 / 1000, font_size=20, s=60, log_min_val=-8, max_area_val=35):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #

    pixel = pow(pixel, 2)
    alloys_indices = range(data['data'].__len__())

    if indices is not None:
        alloys_indices = indices

    fig = plt.figure(figsize=(N, M), facecolor='white')
    fig.patch.set_facecolor('white')
    ax = plt.axes()
    ax.set_facecolor("white")

    plt.rcParams.update({'font.size': font_size})

    markers = ['v', 's', 'D', 'd', 'p', '*', 'P']
    colors = ['orange', 'red', 'blue', 'green', 'indigo']

    legend_a = []

    for k, i in tqdm(enumerate(alloys_indices)):
        cnts = data['data'][i]['contours_series']
        polygon_areas = np.array([cv2.contourArea(np.array(cnt)) for cnt in cnts])

        # print('middle')
        images_num = data['data'][i]['linear_approx_plot'][0]['a_beams'][0].__len__()

        co_fraction = np.sum(polygon_areas) / (pow(1535, 2) * images_num)

        polygon_areas, areas_set, areas_dens_curve = stats_preprocess(np.sqrt(polygon_areas * pixel), step)

        areas_dens_curve /= np.sum(areas_dens_curve)

        areas_dens_curve = np.log(areas_dens_curve)

        areas_dens_curve = [h for h in areas_dens_curve if h > log_min_val]
        areas_set = areas_set[:len(areas_dens_curve)]

        polygon_areas = [area for area in polygon_areas if area < areas_set[-1]]

        polygon_areas = [area for area in polygon_areas if area < max_area_val]
        areas_set = [area for area in areas_set if area < max_area_val]
        areas_dens_curve = areas_dens_curve[:len(areas_set)]

        polygon_mean = np.mean(polygon_areas)
        polygon_std = np.std(polygon_areas)

        plt.scatter(areas_set, areas_dens_curve, marker=markers[i], color=colors[i], s=s)

        areas_dens_curve = np.array(areas_dens_curve)
        areas_set = np.array(areas_set)

        (x_pred, y_pred), k, b, angle, score = lin_regr_approx(areas_set[2:35].reshape(-1, 1),
                                                               areas_dens_curve[2:35].reshape(-1, 1))

        plt.plot(x_pred, y_pred, color=colors[i])

        legend_a.append(data['data'][i]['name'] + ' ' + data['data'][i][
            'type'] + f'\nсредний радиус={round(polygon_mean, 2)} $мкм$\nstd={np.round(polygon_std, 2)}\nдоля кобальта ={np.round(co_fraction, 2)}'
                      f'\nk= {round(k, 2)}\nscore= {round(score, 2)}\nшаг радиуса={step}')
        legend_a.append('-------')

    plt.legend(legend_a)
    plt.ylabel('p ( x )')

    plt.xlabel('Эффективный радиус, $мкм$')
    # ax2.legend(legend_a, loc=1)

    # plt.xticks(np.arange(0,51,5))
    # ax2.set_xticks(np.arange(0,12,3))
    # plt.xlim(0,50)
    # plt.ylim(0,0.15)
    # plt.ylim(-8,0)

    if save:
        plt.savefig(f'polygons_radius_{saved_image_name}_step_{step}.png', bbox_inches='tight')

    plt.show()


In [ ]:
%%time
N = 25
M = 14

# save=False
save = True
step = 1

saved_image_name = f'original'
polygons_radius_all(data, saved_image_name, step, N, M, indices=[3, 0, 2], save=save, start=0, log_min_val=-10,
                    max_area_val=100)
# polygons_area_all(data,saved_image_name, step, N, M, indices=[3,0,2],save=save,end=None)
# polygons_area_all(path,saved_image_name, step, N, M, indices=None,save=save)
